In [1]:
!pip install witwidget-gpu
!jupyter nbextension install --py --symlink --sys-prefix witwidget
!jupyter nbextension enable --py --sys-prefix witwidget

2021-07-18 23:00:41.343633: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-07-18 23:00:41.343687: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Installing c:\python\python39\lib\site-packages\witwidget\static -> wit-widget
- Validating: ok

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable witwidget --py --sys-prefix
    
2021-07-18 23:00:45.308853: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-07-18 23:00:45.308897: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Enabling notebook extension wit-widget/extension...
      

In [2]:
!pip install pandas

In [3]:
import witwidget

from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [4]:
import csv
import json

jsonFilePath = open('adultData.json', 'w')

data={}

columns = [ 
            "age",
            "workclass", 
            "fnlwgt", 
            "education", 
            "education-num", 
            "marital-status", 
            "occupation", 
            "relationship", 
            "race" , 
            "sex", 
            "capital-gain", 
            "capital-loss", 
            "hours-per-week",
            "native-country",
            ">50K, <=50K"]


with open("adult.data") as csv_file:
    csv_reader = csv.DictReader(csv_file,columns)
    count = 0
    for row in csv_reader:       
            json.dump(row, jsonFilePath)
            jsonFilePath.write('\n')

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import IPython.display as display
import pandas as pd

adults_train = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", 
                    names=["age","workclass","fnlwgt","education","education-num","marital-status", 
                            "occupation","relationship","race","sex","capital-gain","capital-loss", 
                            "hours-per-week","native-country","50K"])

adults_train.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,50K
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
adults_features = adults_train.copy()
adults_labels = adults_features.pop('age')

In [7]:
# Create a symbolic input
input = tf.keras.Input(shape=(), dtype=tf.float32)

# Do a calculation using is
result = 2*input + 1

# the result doesn't have a value
result

<KerasTensor: shape=(None,) dtype=float32 (created by layer 'tf.__operators__.add')>

In [8]:
calc = tf.keras.Model(inputs=input, outputs=result)

In [9]:
inputs = {}

for name, column in adults_features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

{'workclass': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'workclass')>,
 'fnlwgt': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fnlwgt')>,
 'education': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'education')>,
 'education-num': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'education-num')>,
 'marital-status': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'marital-status')>,
 'occupation': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'occupation')>,
 'relationship': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'relationship')>,
 'race': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'race')>,
 'sex': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'sex')>,
 'capital-gain': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'capital-gain')>,
 'capital-loss': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'capital-loss'

In [10]:
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = preprocessing.Normalization()
norm.adapt(np.array(adults_train[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

all_numeric_inputs

<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'normalization')>

In [11]:
calc = tf.keras.Model(inputs=input, outputs=result)


In [12]:
print(calc(1).numpy())
print(calc(2).numpy())

3.0
5.0


In [13]:
inputs = {}

for name, column in adults_features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

{'workclass': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'workclass')>,
 'fnlwgt': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fnlwgt')>,
 'education': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'education')>,
 'education-num': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'education-num')>,
 'marital-status': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'marital-status')>,
 'occupation': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'occupation')>,
 'relationship': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'relationship')>,
 'race': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'race')>,
 'sex': <KerasTensor: shape=(None, 1) dtype=string (created by layer 'sex')>,
 'capital-gain': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'capital-gain')>,
 'capital-loss': <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'capital-loss'

In [14]:
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = preprocessing.Normalization()
norm.adapt(np.array(adults_train[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)

all_numeric_inputs

<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'normalization_1')>

In [15]:
preprocessed_inputs = [all_numeric_inputs]

In [16]:
for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue

  lookup = preprocessing.StringLookup(vocabulary=np.unique(adults_features[name]))
  one_hot = preprocessing.CategoryEncoding(max_tokens=lookup.vocab_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

In [17]:
preprocessed_inputs_cat = layers.Concatenate()(preprocessed_inputs)

adults_preprocessing = tf.keras.Model(inputs, preprocessed_inputs_cat)

tf.keras.utils.plot_model(model = adults_preprocessing , rankdir="LR", dpi=72, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [18]:
adults_features_dict = {name: np.array(value) 
                         for name, value in adults_features.items()}


In [19]:
features_dict = {name:values[:1] for name, values in adults_features_dict.items()}
adults_preprocessing(features_dict)

<tf.Tensor: shape=(1, 127), dtype=float32, numpy=
array([[-1.0636126 ,  1.1347412 ,  0.14845309, -0.21665974, -0.03542657,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
 

In [20]:
def adult_model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
    layers.Dense(64),
    layers.Dense(1)
  ])

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.losses.BinaryCrossentropy(from_logits=True),
                optimizer=tf.optimizers.Adam())
  return model

adult_model = adult_model(adults_preprocessing, inputs)

In [21]:
adult_model.fit(x=adults_features_dict, y=adults_labels, epochs=10)

Epoch 1/10
1018/1018 [==============================] - 3s 2ms/step - loss: -19843.7305
Epoch 2/10
1018/1018 [==============================] - 2s 2ms/step - loss: -129663.4688
Epoch 3/10
1018/1018 [==============================] - 2s 2ms/step - loss: -323662.5000
Epoch 4/10
1018/1018 [==============================] - 2s 2ms/step - loss: -584839.2500
Epoch 5/10
1018/1018 [==============================] - 2s 2ms/step - loss: -905072.8750
Epoch 6/10
1018/1018 [==============================] - 2s 2ms/step - loss: -1280199.3750
Epoch 7/10
1018/1018 [==============================] - 2s 2ms/step - loss: -1708130.6250
Epoch 8/10
1018/1018 [==============================] - 2s 2ms/step - loss: -2187771.7500
Epoch 9/10
1018/1018 [==============================] - 2s 2ms/step - loss: -2718537.7500
Epoch 10/10
1018/1018 [==============================] - 2s 2ms/step - loss: -3299982.0000


In [22]:
adult_model.save('adultTestModel')
reloaded = tf.keras.models.load_model('adultTestModel')

INFO:tensorflow:Assets written to: adultTestModel\assets


INFO:tensorflow:Assets written to: adultTestModel\assets


In [26]:
features_dict = {name:values[:1] for name, values in adults_features_dict.items()}

before = adult_model(features_dict)
after = reloaded(features_dict)
assert (before-after)<1e-3
print(before)
print(after)

tf.Tensor([[98946.914]], shape=(1, 1), dtype=float32)
tf.Tensor([[98946.914]], shape=(1, 1), dtype=float32)


In [27]:
import itertools

def slices(features):
  for i in itertools.count():
    # For each feature take index `i`
    example = {name:values[i] for name, values in features.items()}
    yield example

In [29]:
for example in slices(adults_features_dict):
  for name, value in example.items():
    print(f"{name:19s}: {value}")
  break

workclass          :  State-gov
fnlwgt             : 77516
education          :  Bachelors
education-num      : 13
marital-status     :  Never-married
occupation         :  Adm-clerical
relationship       :  Not-in-family
race               :  White
sex                :  Male
capital-gain       : 2174
capital-loss       : 0
hours-per-week     : 40
native-country     :  United-States
50K                :  <=50K


In [30]:
features_ds = tf.data.Dataset.from_tensor_slices(adults_features_dict)


In [31]:
for example in features_ds:
  for name, value in example.items():
    print(f"{name:19s}: {value}")
  break

workclass          : b' State-gov'
fnlwgt             : 77516
education          : b' Bachelors'
education-num      : 13
marital-status     : b' Never-married'
occupation         : b' Adm-clerical'
relationship       : b' Not-in-family'
race               : b' White'
sex                : b' Male'
capital-gain       : 2174
capital-loss       : 0
hours-per-week     : 40
native-country     : b' United-States'
50K                : b' <=50K'


In [32]:
adults_ds = tf.data.Dataset.from_tensor_slices((adults_features_dict, adults_labels))

In [33]:
adults_batches = adults_ds.shuffle(len(adults_labels)).batch(32)

In [34]:
adult_model.fit(adults_batches, epochs=5)


Epoch 1/5
1018/1018 [==============================] - 2s 2ms/step - loss: -3931652.0000
Epoch 2/5
1018/1018 [==============================] - 2s 2ms/step - loss: -4613335.0000
Epoch 3/5
1018/1018 [==============================] - 2s 2ms/step - loss: -5344431.5000
Epoch 4/5
1018/1018 [==============================] - 2s 2ms/step - loss: -6124911.0000
Epoch 5/5
1018/1018 [==============================] - 2s 2ms/step - loss: -6954987.0000
